# Sprint Analysis Overview

This analysis explores player behavior, engagement, and interaction patterns within the Sprint game environment. Using detailed event and session data, we aim to uncover insights such as player playtime distributions, encounter dynamics between teams, revenue contributions by geography, and performance metrics at both individual and aggregated levels.

The data pipeline is built with dbt, transforming raw event logs into curated fact and dimension tables, and further into specialized marts optimized for business queries. This structure enables analysts to efficiently answer critical questions about player activity, session encounters, and monetization trends over time.

The following report presents key metrics and aggregated views designed to support strategic decision-making and game experience enhancements.

In [34]:
import duckdb
import pandas as pd
from pathlib import Path

db_path = Path('../data/synthetic.duckdb').resolve()
conn = duckdb.connect(database=str(db_path), read_only=False)

def run_query(sql):
    df = conn.execute(sql).fetchdf()
    return df

In [9]:
arch = run_query('select * from information_schema.tables')[['table_schema', 'table_name']]
print(f'This project generated {arch.table_name.nunique()} tables in {arch.table_schema.nunique()} schemas according to a medallion paradigm')
arch

This project generated 16 tables in 4 schemas according to a medallion paradigm


,table_schema,table_name
0,sprint_dim,dim_players
1,sprint_dim,dim_products
2,sprint_mart,country_monthly_playtime
3,sprint_mart,country_weekly_revenue
4,sprint_mart,encounter_summary_daily
5,sprint_mart,player_activity_daily
6,sprint_mart,player_consecutive_days_monthly
7,sprint_mart,player_stats_lifetime
8,sprint_mart,session_close_encounters_daily
9,sprint_raw,event_session


The Python generators created 1,000 unique sample players and simulated their sign-in patterns over the course of a year. They modeled session activity with matchmaking, logged all player actions into the database, and built a concise analytics layer in mart tables for streamlined, clean analysis.

## Questions
### 1. Average play time on 2025-07-07 for any player (e.g., pick the first player in the dataset)


In [11]:
select_date = '2025-07-07'
print(f"Average play time of a player on {select_date}")

# Pick one player who played that day
player_sql = f"""
SELECT DISTINCT player_id
FROM sprint_mart.player_activity_daily
WHERE calendar_date = DATE '{select_date}'
LIMIT 1
"""
player_id = conn.execute(player_sql).fetchone()[0]
print(f"Selected player: {player_id}")

# Query that player’s playtime on the date from the mart
query_1 = f"""
SELECT 
    player_id,
    calendar_date,
    total_play_time_seconds
FROM sprint_mart.player_activity_daily
WHERE player_id = '{player_id}'
  AND calendar_date = DATE '{select_date}'
"""
df1 = run_query(query_1)

played_seconds = df1['total_play_time_seconds'].iloc[0] if not df1.empty else 0
print(f"On {select_date}, player {player_id} played for {played_seconds} seconds.\n")


Average play time of a player on 2025-07-07
Selected player: 0293
On 2025-07-07, player 0293 played for 553.0 seconds.



### 2. Week with the highest volume of players

In [12]:
print("2. Week with highest volume of unique players")
num_rows = 1

query_2 = f"""
SELECT 
    date_trunc('week', calendar_date::timestamp) AS year_week,
    COUNT(DISTINCT player_id) AS unique_players
FROM sprint_mart.player_activity_daily
GROUP BY year_week
ORDER BY unique_players DESC
LIMIT {num_rows}
"""

df2 = run_query(query_2)
print(f"The week starting {df2['year_week'].iloc[0].date()} had the highest number of unique players: {df2['unique_players'].iloc[0]}.\n")


2. Week with highest volume of unique players
The week starting 2025-07-28 had the highest number of unique players: 788.



### 3. Country with the most playtime for June 2025

In [13]:
print("3. Country with the most time played for June 2025")
query_3 = """
SELECT 
    country,
    total_play_time_seconds
FROM sprint_mart.country_monthly_playtime
WHERE year_month = DATE '2025-06-01'
ORDER BY total_play_time_seconds DESC
LIMIT 1
"""
df3 = run_query(query_3)
print(f"In June 2025, {df3['country'].iloc[0]} had the most total play time with {df3['total_play_time_seconds'].iloc[0]} seconds.\n")


3. Country with the most time played for June 2025
In June 2025, US had the most total play time with 750915.0 seconds.



### 4. Player-specific statistics (using selected player above)

In [15]:
print("4a. Player lifetime kill/death average")

query_4a = f"""
SELECT
    player_id,
    kill_death_ratio
FROM sprint_mart.player_activity_daily
WHERE player_id = '{player_id}'
ORDER BY calendar_date DESC
LIMIT 1
"""
df4a = run_query(query_4a)
kd_ratio = df4a['kill_death_ratio'].iloc[0] if not df4a.empty else None
print(f"Player {player_id} lifetime kill/death ratio is {kd_ratio}.\n")


4a. Player lifetime kill/death average
Player 0293 lifetime kill/death ratio is 12.0.



In [17]:
# 4b. Month with most consecutive play days
query_4b = f"""
SELECT 
    year_month,
    max_consecutive_days_played
FROM sprint_mart.player_consecutive_days_monthly
WHERE player_id = '{player_id}'
ORDER BY max_consecutive_days_played DESC
LIMIT 1
"""

df4b = run_query(query_4b)

if not df4b.empty:
    month = df4b['year_month'].iloc[0].date()
    days = df4b['max_consecutive_days_played'].iloc[0]
    print(f"Player {player_id} played the most consecutive days ({days}) in {month}.\n")
else:
    print(f"No consecutive play days data available for player {player_id}.\n")


Player 0293 played the most consecutive days (2) in 2025-08-01.



### 5. Country with the most revenue per week?

In [18]:
print("5. Country that has driven the most revenue per week")

query_5 = f"""
SELECT 
    country,
    year_week,
    total_revenue
FROM sprint_mart.country_weekly_revenue
ORDER BY total_revenue DESC
LIMIT 1
"""

df5 = run_query(query_5)

if not df5.empty:
    country = df5['country'].iloc[0]
    week_start = df5['year_week'].iloc[0].date()
    revenue = df5['total_revenue'].iloc[0]
    print(f"The country with the highest revenue was {country} in the week starting {week_start} with revenue of {revenue}.\n")
else:
    print("No revenue data available.\n")


5. Country that has driven the most revenue per week
The country with the highest revenue was MX in the week starting 2025-01-20 with revenue of 8935.94.



### 6. Close encounters per session per day in June 2025

In [27]:
print("6. Close encounters per session by day in June 2025")

query_6 = f"""
SELECT 
    session_id,
    calendar_day,
    sum(daily_encounter_count) AS total_close_encounters
FROM sprint_mart.encounter_summary_daily
WHERE calendar_day >= DATE '2025-06-01'
  AND calendar_day < DATE '2025-07-01'
GROUP BY session_id, calendar_day
ORDER BY session_id, calendar_day
"""

df6 = run_query(query_6).sort_values(by='calendar_day', ascending=True)
df6

6. Close encounters per session by day in June 2025


,session_id,calendar_day,total_close_encounters
171,42d3f187-8c54-4c0e-bf20-8221f846cd64,2025-06-01,9.0
531,c2a04229-8998-4b59-9966-e9fdc739ba3e,2025-06-01,1.0
648,f127ab10-b22d-4211-a500-0ff5e0558228,2025-06-01,5.0
317,778972d2-d64b-4322-b863-d6e9a0eb6a16,2025-06-01,10.0
507,baa48260-e6cc-49ed-9178-eff855a586b8,2025-06-01,6.0
...,...,...,...
83,1d780a49-4bc6-4360-aff6-fcf131f21228,2025-06-30,1.0
299,71b386ca-dacd-405f-9dae-a9e2c871b2a0,2025-06-30,3.0
60,1854d927-0dbd-4030-8f68-178e27359cc3,2025-06-30,3.0
323,79345547-b9da-418d-978d-a2e6e95d458d,2025-06-30,7.0


### Estimated Table Size and Growth Projection

We generated data for 1,000 unique players, simulating up to 1 session per day per sign-on, over the course of one year.

Looking ahead, we aim to scale this to approximately **1 million daily active players** within the next year.

#### Baseline Estimates for 1,000 Players in 2025:

- **~12.7K distinct sessions** (assuming partial daily activity)  
- **Up to 10 sessions per player per day** (upper bound; actual player sign-ins vary)  
- **~127K realistic sessions** based on expected player activity  
- **~30 heartbeats per session**  
- Resulting in approximately **3.8 million heartbeat events**  

#### Growth Projection for 1 Million Players in 2026:

Scaling linearly by player count, we expect roughly:

- **~12.7 million distinct sessions**  
- **~127 million realistic sessions**  
- Approximately **3 billion heartbeat events**  

This represents a **1,000× increase** in data volume over one year.

---

### Scaling Considerations and Solutions

Handling this scale requires careful planning and architectural choices:

- **Data Partitioning and Clustering:** Organize tables by time (e.g., monthly partitions) and player or session identifiers to optimize query performance and reduce scan costs.  
- **Incremental Processing:** Use incremental models and ETL pipelines (e.g., with dbt) to process only new or changed data, minimizing resource use and speeding up data refreshes.  
- **Data Archiving:** Archive or purge older data no longer needed for real-time analytics to manage storage and maintain performance.  
- **Cloud Data Warehousing:** Leverage scalable, cloud-native warehouses (Snowflake, BigQuery, Redshift) that elastically handle petabyte-scale data and offer robust performance optimizations.  
- **Pre-aggregated Marts:** Build specialized aggregate tables and materialized views for common queries, reducing the need to scan large raw tables repeatedly.  
- **Monitoring and Cost Control:** Implement monitoring for query performance and cost, enabling proactive optimization and budget management.

By combining these approaches, the platform can efficiently scale to millions of players while supporting fast, insightful analytics.
